In [41]:
from text_preprocessing import spacy_preprocessing
import spacy
nlp = spacy.blank("fr")

In [42]:
def find_swift(text):
    if "entity_ruler" not in nlp.pipe_names:
        ruler = nlp.add_pipe("entity_ruler")
    else:
        ruler= nlp.get_pipe("entity_ruler")
    text = spacy_preprocessing(text)
    # we use spacy s entity ruler for matching
    patterns =[
        ## uppercases will be lower after preprocessing
         #matches :code(optional) all words wich their lower case form is swift followed  by  space or punct and the code
        {"label":"Code Swift","pattern":[{"LOWER":"code","OP":"?"},{"LOWER":"swift"},{'IS_SPACE':True,'OP':'?'},{"IS_PUNCT":True,"OP":"?"},{"TEXT": {"REGEX": "[A-Za-z0-9]{8,11}"}}]},
        #matches : Swift(e)(with some possible typos) tied to the code
        {"label":"Code Swift","pattern":[{"TEXT": {"REGEX": "(swift)[e]?[A-Za-z0-9]{8,11}"}}]},
        #matches abbreviations:
        {"label":"Code Swift","pattern":[{"TEXT": {"REGEX": "^(sft|st)[e]?[A-Za-z0-9]{8,11}"}}]},
        # matches swift(any special character)code
        {"label":"Code Swift","pattern":[{"TEXT": {"REGEX": "^(swift|sft|st)[e]?[!@#$%^&*()_+\-=\[\]{};':'\\|,.<>\/?][A-Za-z0-9]{8,11}"}}]}
        ]
    ruler.add_patterns(patterns)
    tokens = nlp(text)
    return [ent[-1] for ent in tokens.ents]


In [43]:
file = 'C:/Users/PC2/Downloads/test-ex.txt'
f=open(file,'r')
data = f.read()
find_swift(data)



[agilamcxx, creschzz80a]

In [44]:
def check_imo_funct(imo_code):
    if len(str(imo_code))!=7:
        return False
    else:
        digits = [int(x) for x in str(imo_code)]
        s=0
        for i in range(len(digits)-1):
            s+= digits[i]*(len(digits)-i)
        if s%10==digits[6]:
            return True
        else :
            return False



In [45]:
def find_imo(text):    
    if "entity_ruler" not in nlp.pipe_names:
        ruler = nlp.add_pipe("entity_ruler")
    else:
        ruler= nlp.get_pipe("entity_ruler")
    text = spacy_preprocessing(text)
    # we use spacy s entity ruler for matching
    patterns =[
        #matches : imo 2313... or imo : 2313... ...
        {"label":"Code IMO","pattern":[{"LOWER":"code","OP":"?"},{"LOWER":"imo"},{'IS_SPACE':True,'OP':'?'},{"IS_PUNCT":True,"OP":"?"},{"TEXT": {"REGEX": "[0-9]{7}"}}]},
        #matches imo2343..., also 2313...(alone)
        {"label":"Code IMO","pattern":[{"TEXT": {"REGEX": "^(IMO|imo)?[0-9]{7}"}}]},
         # matches imo(possible typo)(any special character)code
        {"label":"Code IMO","pattern":[{"TEXT": {"REGEX": "^(imo)[a-z]?[!@#$%^&*()_+\-=\[\]{};':'\\|,.<>\/?][0-9]{7}"}}]}
        ]
    ruler.add_patterns(patterns)
    tokens = nlp(text)
    l=[ent[-1] for ent in tokens.ents]
    # we try to extract only the number to check if its really an imo
    possible_imos=[]
    for elt in l :
        num= str(elt)[-7:]
        possible_imos.append(num)
    #check if imo:
    imos = []
    for num in possible_imos:
        if check_imo_funct(num):
            imos.append(num)
    return imos


In [46]:
text='Example: IMO 9619907  After submitting IMO number with SAERCH button current or last known position of the ship will be displayed on the map'
find_imo(text)

c:\Users\PC2\anaconda3\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


['9619907']